In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
%%capture
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!nvcc -V

In [ ]:
!nvidia-smi

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama-2-7b-chat-guanaco"

In [ ]:
dataset = load_dataset(guanaco_dataset, split="train")

In [ ]:
dataset['text'][0]

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0},)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params,
)

In [ ]:
!nvidia-smi

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('fine_tuned_model')

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # به شماره GPU صحیح تغییر دهید

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# مسیر محلی به مدل فاین‌تیون‌شده
fine_tuned_model_path = "fine_tuned_model"

# لود کردن مدل
model = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model_path,
    # device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    offload_folder="./offload",
    load_in_4bit=True
)


# لود کردن توکنایزر
tokenizer = AutoTokenizer.from_pretrained(
    "fine_tuned_model",
    trust_remote_code=True
)

# آماده کردن مدل برای پردازش ورودی
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=50,
    temperature=0.7,
    top_k=10,
    top_p=0.8
)

# آماده برای پرسیدن سؤال
while True:
    user_input = input("پرسش خود را وارد کنید (برای خروج 'exit' را تایپ کنید): ")
    if user_input.lower() == "exit":
        print("خروج از برنامه.")
        break
    prompt = f"<s>[INST] {user_input} [/INST]"
    response = pipe(prompt)
    print("پاسخ مدل:", response[0]['generated_text'])

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")

# from peft import PeftModel

# model = AutoModelForCausalLM.from_pretrained(
#     "fine_tuned_model",
#     torch_dtype=torch.float16,
#     # device_map=
# )

# # model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=1)
# model = PeftModel.from_pretrained(model, "fine_tuned_model")
# # model = model.merge_and_unload()

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "hello, how are you?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=20)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
print('hello, welcome to our llama model, if you want to exit the app enter: exit')
while True:
    prompt = str(input('enter your prompt: '))
    
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    print(result[0]['generated_text'])

    if prompt == "exit":
        break